Ingest Matern Hospital Health Data Source File

In [0]:
processName = dbutils.widgets.get('prm_processName')

nextSourceFileDateSql = f"""SELECT NVL(MAX(PROCESSED_FILE_TABLE_DATE)+1,'2025-06-01')  as NEXT_SOURCE_FILE_DATE FROM materncare_anlytics.processlogs.deltalakehouse_process_run 
WHERE PROCESS_NAME = '{processName}' and PROCESS_STATUS='Completed'"""


nextSourceFileDateDF = spark.sql(nextSourceFileDateSql)
display(nextSourceFileDateDF)

In [0]:
MaternHosSourceLayerName= "materncare-analytics"
MaternHosSourceStorageName= "materncaredatalakehouse"
MaternHosSourceFileName= "Maternal_Health-Hospital.csv"


MaternHosSinkLayerName= "bronze"
MaternHosSinkStorageName= "materncaredatalakehouse"
MaternHosSinkStorageFolderName="Matern-Hospital_Health"




In [0]:
%python
import pandas as pds

In [0]:
%python
%pip install fsspec

from pyspark.sql.functions import current_timestamp
MaternHosSourceFolderPath = f"abfss://{MaternHosSourceLayerName}@{MaternHosSourceStorageName}.dfs.core.windows.net/{MaternHosSourceFileName}"
print(MaternHosSourceFolderPath)

MaternHospitalSparkDF = spark.read.format('csv').option('header', 'true').load(MaternHosSourceFolderPath); MaternHospitalPandaDF = MaternHospitalSparkDF.toPandas()
print(MaternHospitalPandaDF)

In [0]:
MaternHospitalSparkDFWrite =  spark.createDataFrame(MaternHospitalPandaDF)

In [0]:
from pyspark.sql.functions import current_timestamp
MaternHosSinkFolderPath = f"abfss://{MaternHosSinkLayerName}@{MaternHosSinkStorageName}.dfs.core.windows.net/{MaternHosSinkStorageFolderName}"
print(MaternHosSinkFolderPath)

(
    MaternHospitalSparkDFWrite
    .withColumn("source_file_load_date",current_timestamp())
    .write
    .mode("append")
    .option("header","true")
    .csv(MaternHosSinkFolderPath)

)

In [0]:
processFileDate = nextSourceFileDateDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE']
processStatus ='Completed'

processInsertSql = f""" INSERT INTO materncare_anlytics.processlogs.deltalakehouse_process_run(PROCESS_NAME,PROCESSED_FILE_TABLE_DATE,PROCESS_STATUS) VALUES('{processName}','{processFileDate}','{processStatus}')"""

spark.sql(processInsertSql)